<a href="https://colab.research.google.com/github/justorfc/Estadistica_Aplicada/blob/main/Modelaci%C3%B3n_hacia_adelante_y_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regresión Lineal Múltiple forward**

A continuación se presenta una introducción detallada sobre la modelación para la regresión lineal múltiple utilizando `statsmodels` en Python, aplicando los métodos de selección de variables hacia adelante, hacia atrás y ambos combinados, utilizando el dataset `Boston`.

# **NOTA: A continuación se muestran algunos código que no han sido probados y depurados, se puede y debería copiar el contenido de esta celda de texto en un Notebook nuevo y pegar todo el código que aquí se muestra en celdas de código correspondientes y revisarlo. Sin embargo en las celdas de código que le siguen a esta celda de texto están los códigos totalmente funcionales que realizan Modelación forward con el dataset Boston del paquete MASS de R en Python**

# Introducción a la Regresión Lineal Múltiple y Selección de Variables con `statsmodels` en Python

La regresión lineal múltiple es una técnica estadística que nos permite modelar la relación entre una variable dependiente continua y varias variables independientes. Esta técnica es útil en diversas áreas, incluyendo la economía, la biología, y las ciencias de datos, donde se requiere entender y predecir el comportamiento de una variable en función de otras.

En la práctica, no todas las variables independientes disponibles son necesariamente útiles para nuestro modelo. Algunas pueden no tener una relación significativa con la variable dependiente, o pueden estar altamente correlacionadas con otras variables, lo que puede afectar la estabilidad y la interpretabilidad del modelo. Para abordar este problema, se emplean métodos de selección de variables que permiten identificar un subconjunto óptimo de variables independientes.

## Selección de Variables con `statsmodels`

El paquete `statsmodels` en Python ofrece funciones avanzadas para la modelación estadística, incluida la regresión lineal múltiple. Para la selección de variables, utilizaremos una combinación de técnicas manuales y métricas como el criterio de información de Akaike (AIC). El AIC es una medida que considera tanto la bondad de ajuste del modelo como su complejidad, penalizando por el número de parámetros para evitar el sobreajuste.

### Métodos de Selección de Variables

1. **Modelación hacia Adelante (Forward Selection)**: Comienza con un modelo nulo (sin variables independientes) y agrega variables una por una, seleccionando en cada paso la variable que más reduce el AIC, hasta que no se puede reducir más el AIC con la adición de nuevas variables.

2. **Modelación hacia Atrás (Backward Selection)**: Comienza con el modelo completo (con todas las variables independientes) y elimina variables una por una, seleccionando en cada paso la variable cuya eliminación reduce más el AIC, hasta que no se puede reducir más el AIC eliminando más variables.

3. **Modelación de Ambos Sentidos (Stepwise Selection)**: Combina los dos métodos anteriores, permitiendo tanto la adición como la eliminación de variables en cada paso. Este método intenta encontrar un equilibrio óptimo entre la simplicidad del modelo y su capacidad de predicción.

### Objetivos del Notebook

En este notebook, se realizará un análisis práctico utilizando `statsmodels` para aplicar los métodos de selección de variables mencionados. Los objetivos específicos son:

1. **Implementar y comparar la modelación hacia adelante, hacia atrás y de ambos sentidos**.
2. **Evaluar el desempeño de los modelos seleccionados utilizando métricas de ajuste**.
3. **Interpretar los resultados y seleccionar el modelo más adecuado basado en el AIC, R² y R² ajustado**.

### Preparación del Entorno de Trabajo

Asegúrate de tener instalados y cargados los paquetes necesarios en tu entorno de Python. El dataset `Boston` se cargará utilizando `statsmodels`.

```python
# Instalación de los paquetes necesarios
!pip install statsmodels pandas numpy

# Importación de los paquetes necesarios
import statsmodels.api as sm
import pandas as pd
import numpy as np
from itertools import combinations

# Carga del dataset Boston
boston_data = sm.datasets.get_rdataset('Boston', package='MASS').data
```

### Implementación de la Modelación y Selección de Variables

#### 1. El Mejor Modelo Global

```python
# Definición de las variables independientes (X) y la variable dependiente (y)
X = boston_data.drop(columns='medv')  # 'medv' es la variable dependiente
y = boston_data['medv']

# Añadir una constante al modelo (intercepto)
X = sm.add_constant(X)

# Ajustar el modelo completo
model = sm.OLS(y, X).fit()

# Mostrar el resumen del modelo
print(model.summary())
```

#### 2. Los Mejores Modelos y sus Coeficientes

```python
# Forward Selection
def forward_selection(data, target):
    initial_features = []
    best_features = initial_features[:]
    while True:
        remaining_features = list(set(data.columns) - set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(pd.DataFrame(data[best_features + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < 0.05:
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

best_features_forward = forward_selection(X.drop(columns='const'), y)
print("Mejores variables (Forward Selection):", best_features_forward)

# Ajustar el modelo con las mejores variables
X_forward = sm.add_constant(X[best_features_forward])
model_forward = sm.OLS(y, X_forward).fit()
print(model_forward.summary())
```

#### 3. Los Mejores Modelos con sus Variables y sus Coeficientes

```python
# Backward Selection
def backward_selection(data, target):
    features = list(data.columns)
    while True:
        pvals = sm.OLS(target, sm.add_constant(pd.DataFrame(data[features]))).fit().pvalues[1:]
        max_p_value = pvals.max()
        if max_p_value >= 0.05:
            excluded_feature = pvals.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

best_features_backward = backward_selection(X.drop(columns='const'), y)
print("Mejores variables (Backward Selection):", best_features_backward)

# Ajustar el modelo con las mejores variables
X_backward = sm.add_constant(X[best_features_backward])
model_backward = sm.OLS(y, X_backward).fit()
print(model_backward.summary())
```

#### 4. La Modelación con AIC, R² y R² Ajustado

```python
# Comparación de modelos con AIC, R² y R² ajustado
models = {
    'Global': model,
    'Forward Selection': model_forward,
    'Backward Selection': model_backward
}

for name, mdl in models.items():
    print(f"{name} - AIC: {mdl.aic}, R²: {mdl.rsquared}, R² ajustado: {mdl.rsquared_adj}")
```

## Mejor modelo global

In [ ]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

# Cargar el dataset Boston desde statsmodels
boston_data = sm.datasets.get_rdataset('Boston', package='MASS').data

# Lista de nombres de variables (características)
variables = boston_data.columns[:-1].tolist()

# Crear todos los posibles modelos con una sola variable
models = {}
for var in variables:
    model = sm.OLS(boston_data['medv'], sm.add_constant(boston_data[var])).fit()
    models[var] = {'model': model, 'AIC': model.aic}

# Encontrar el modelo más simple (con una variable) con el menor AIC
best_model = min(models.items(), key=lambda x: x[1]['AIC'])

# Iterar para agregar una variable más a los modelos existentes
num_variables = 2
while num_variables <= len(variables):
    # Crear todos los modelos con num_variables variables
    combos = combinations(variables, num_variables)
    for combo in combos:
        combo_name = '_'.join(combo)
        combo_vars = [boston_data[var] for var in combo]
        combo_vars = sm.add_constant(pd.concat(combo_vars, axis=1))
        model = sm.OLS(boston_data['medv'], combo_vars).fit()
        models[combo_name] = {'model': model, 'AIC': model.aic}

    # Encontrar el mejor modelo con num_variables variables
    best_model = min(models.items(), key=lambda x: x[1]['AIC'])

    num_variables += 1

# Mostrar el mejor modelo y su AIC
print(f"Mejor modelo encontrado: {best_model[0]} con AIC: {best_model[1]['AIC']}")


## Mejores Modelos de 1 a n Variables y Modelo Global

In [ ]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

# Cargar el dataset Boston desde statsmodels
boston_data = sm.datasets.get_rdataset('Boston', package='MASS').data

# Lista de nombres de variables (características)
variables = boston_data.columns[:-1].tolist()

# Crear todos los posibles modelos con una sola variable
models = {}
for var in variables:
    model = sm.OLS(boston_data['medv'], sm.add_constant(boston_data[var])).fit()
    models[var] = {'model': model, 'AIC': model.aic}

# Encontrar el modelo más simple (con una variable) con el menor AIC
best_model = min(models.items(), key=lambda x: x[1]['AIC'])

# Imprimir el mejor modelo con una variable y su AIC
print(f"Mejor modelo con una variable: {best_model[0]} con AIC: {best_model[1]['AIC']}")
print()

# Iterar para agregar una variable más a los modelos existentes
num_variables = 2
while num_variables <= len(variables):
    # Crear todos los modelos con num_variables variables
    combos = combinations(variables, num_variables)
    for combo in combos:
        combo_name = '_'.join(combo)
        combo_vars = [boston_data[var] for var in combo]
        combo_vars = sm.add_constant(pd.concat(combo_vars, axis=1))
        model = sm.OLS(boston_data['medv'], combo_vars).fit()
        models[combo_name] = {'model': model, 'AIC': model.aic}

    # Encontrar el mejor modelo con num_variables variables
    best_model = min(models.items(), key=lambda x: x[1]['AIC'])
    print(f"Mejor modelo con {num_variables} variables: {best_model[0]} con AIC: {best_model[1]['AIC']}")

    num_variables += 1
    print()

# Mostrar el mejor modelo global y su AIC
print(f"Mejor modelo encontrado globalmente: {best_model[0]} con AIC: {best_model[1]['AIC']}")


## Mejores Modelos y sus Coeficientes

Se puede ajustar el código anterior para que además de mostrar los nombres de las variables en cada modelo, también imprima los coeficientes correspondientes. Aquí está el código modificado:

In [ ]:
# Claro, puedo ajustar el código para que además de mostrar los nombres de las variables en cada modelo, también imprima los coeficientes correspondientes. Aquí está el código modificado:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

# Cargar el dataset Boston desde statsmodels
boston_data = sm.datasets.get_rdataset('Boston', package='MASS').data

# Lista de nombres de variables (características)
variables = boston_data.columns[:-1].tolist()

# Crear todos los posibles modelos con una sola variable
models = {}
for var in variables:
    model = sm.OLS(boston_data['medv'], sm.add_constant(boston_data[var])).fit()
    models[var] = {'model': model, 'AIC': model.aic, 'coefficients': [model.params['const'], model.params[var]]}

# Encontrar el modelo más simple (con una variable) con el menor AIC
best_model = min(models.items(), key=lambda x: x[1]['AIC'])

# Imprimir el mejor modelo con una variable, sus coeficientes y su AIC
print(f"Mejor modelo con una variable: {best_model[0]} con AIC: {best_model[1]['AIC']}")
print(f"Coeficientes: Intercepto: {best_model[1]['coefficients'][0]}, {best_model[0]}: {best_model[1]['coefficients'][1]}")
print()

# Iterar para agregar una variable más a los modelos existentes
num_variables = 2
while num_variables <= len(variables):
    # Crear todos los modelos con num_variables variables
    combos = combinations(variables, num_variables)
    for combo in combos:
        combo_name = '_'.join(combo)
        combo_vars = [boston_data[var] for var in combo]
        combo_vars = sm.add_constant(pd.concat(combo_vars, axis=1))
        model = sm.OLS(boston_data['medv'], combo_vars).fit()
        models[combo_name] = {'model': model, 'AIC': model.aic, 'coefficients': model.params.tolist()}

    # Encontrar el mejor modelo con num_variables variables
    best_model = min(models.items(), key=lambda x: x[1]['AIC'])
    print(f"Mejor modelo con {num_variables} variables: {best_model[0]} con AIC: {best_model[1]['AIC']}")
    print(f"Coeficientes: {best_model[1]['coefficients']}")

    num_variables += 1
    print()

# Mostrar el mejor modelo global, sus coeficientes y su AIC
print(f"Mejor modelo encontrado globalmente: {best_model[0]} con AIC: {best_model[1]['AIC']}")
print(f"Coeficientes: {best_model[1]['coefficients']}")


## Mejores modelos con sus variables y coeficientes separados

In [ ]:
# El siguiente código muestra los coeficientes de cada modelo junto con sus variables.
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

# Cargar el dataset Boston desde statsmodels
boston_data = sm.datasets.get_rdataset('Boston', package='MASS').data

# Lista de nombres de variables (características)
variables = boston_data.columns[:-1].tolist()

# Crear todos los posibles modelos con una sola variable
models = {}
for var in variables:
    model = sm.OLS(boston_data['medv'], sm.add_constant(boston_data[var])).fit()
    models[var] = {'model': model, 'AIC': model.aic, 'formula': f'medv ~ {var}'}

# Encontrar el modelo más simple (con una variable) con el menor AIC
best_model = min(models.items(), key=lambda x: x[1]['AIC'])

# Imprimir el mejor modelo con una variable, su AIC y coeficientes
print(f"Mejor modelo con una variable: {best_model[1]['formula']} con AIC: {best_model[1]['AIC']}")
print("Coeficientes:")
print(best_model[1]['model'].params)
print()

# Iterar para agregar una variable más a los modelos existentes
num_variables = 2
while num_variables <= len(variables):
    # Crear todos los modelos con num_variables variables
    combos = combinations(variables, num_variables)
    for combo in combos:
        combo_name = '_'.join(combo)
        combo_vars = [boston_data[var] for var in combo]
        combo_vars = sm.add_constant(pd.concat(combo_vars, axis=1))
        model = sm.OLS(boston_data['medv'], combo_vars).fit()
        models[combo_name] = {'model': model, 'AIC': model.aic, 'formula': f'medv ~ {" + ".join(combo)}'}

    # Encontrar el mejor modelo con num_variables variables
    best_model = min(models.items(), key=lambda x: x[1]['AIC'])
    print(f"Mejor modelo con {num_variables} variables: {best_model[1]['formula']} con AIC: {best_model[1]['AIC']}")
    print("Coeficientes:")
    print(best_model[1]['model'].params)

    num_variables += 1
    print()

# Mostrar el mejor modelo global y su AIC
print(f"Mejor modelo encontrado globalmente: {best_model[1]['formula']} con AIC: {best_model[1]['AIC']}")
print("Coeficientes:")
print(best_model[1]['model'].params)


# Modelación con AIC, $R^2$ y $R^2$ Ajustado

El siguiente es el código modificado para incluir los valores de $R^2$ y $R^2$ Ajustado y seleccionar el modelo con el mayor **$R^2$ Ajustado**.

In [ ]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

# Cargar el dataset Boston desde statsmodels
boston_data = sm.datasets.get_rdataset('Boston', package='MASS').data

# Lista de nombres de variables (características)
variables = boston_data.columns[:-1].tolist()

# Crear todos los posibles modelos con una sola variable
models = {}
for var in variables:
    model = sm.OLS(boston_data['medv'], sm.add_constant(boston_data[var])).fit()
    models[var] = {'model': model, 'AIC': model.aic, 'R_squared': model.rsquared, 'R_squared_adj': model.rsquared_adj,
                   'formula': f'medv ~ {var}'}

# Encontrar el modelo más simple (con una variable) con el menor AIC
best_model = min(models.items(), key=lambda x: x[1]['AIC'])

# Imprimir el mejor modelo con una variable, su AIC, R cuadrado y R cuadrado ajustado
print(f"Mejor modelo con una variable: {best_model[1]['formula']} con AIC: {best_model[1]['AIC']}")
print(f"R cuadrado: {best_model[1]['R_squared']}, R cuadrado ajustado: {best_model[1]['R_squared_adj']}")
print("Coeficientes:")
print(best_model[1]['model'].params)
print()

# Iterar para agregar una variable más a los modelos existentes
num_variables = 2
while num_variables <= len(variables):
    # Crear todos los modelos con num_variables variables
    combos = combinations(variables, num_variables)
    for combo in combos:
        combo_name = '_'.join(combo)
        combo_vars = [boston_data[var] for var in combo]
        combo_vars = sm.add_constant(pd.concat(combo_vars, axis=1))
        model = sm.OLS(boston_data['medv'], combo_vars).fit()
        models[combo_name] = {'model': model, 'AIC': model.aic, 'R_squared': model.rsquared, 'R_squared_adj': model.rsquared_adj,
                              'formula': f'medv ~ {" + ".join(combo)}'}

    # Encontrar el mejor modelo con num_variables variables
    best_model = min(models.items(), key=lambda x: x[1]['AIC'])
    print(f"Mejor modelo con {num_variables} variables: {best_model[1]['formula']} con AIC: {best_model[1]['AIC']}")
    print(f"R cuadrado: {best_model[1]['R_squared']}, R cuadrado ajustado: {best_model[1]['R_squared_adj']}")
    print("Coeficientes:")
    print(best_model[1]['model'].params)

    num_variables += 1
    print()

# Mostrar el mejor modelo global y su AIC, R cuadrado y R cuadrado ajustado
print(f"Mejor modelo encontrado globalmente: {best_model[1]['formula']} con AIC: {best_model[1]['AIC']}")
print(f"R cuadrado: {best_model[1]['R_squared']}, R cuadrado ajustado: {best_model[1]['R_squared_adj']}")
print("Coeficientes:")
print(best_model[1]['model'].params)


# Modelación Hacia Atras con R

In [ ]:
load_ext rpy2.ipython

In [ ]:
%%R

library(MASS)
data(Boston)

modelo_completo=lm(medv~., data = Boston)

summary(modelo_completo)

mejor_modelo=stepAIC(modelo_completo)

summary(mejor_modelo)

In [ ]:
%%R

# Tomar las variables del último modelo sugerido pór AIC

library(MASS)
data(Boston)

Modelo_aic = lm(medv ~ , data = Boston)

summary(modelo_aic)

# Conversiones del Notebook de Google Colab

**Nota**: El nombre del notebook no debe contener espacios, observe que el nombre de este notebook no tiene espacios:
```
Convertir_Noteebook_a_PDF_en_Google_Colab.ipynb
```

**Procedimiento:**

1.   Suba el notebook que va a convertir a "/content":
    + 1.1. Por ejemplo descargue este notebook y
    + 1.2. Súbalo a esta sesión de trabajo y luego
2.   Lance en celdas de código las siguientes órdenes:


```
!jupyter nbconvert --to html /content/Convertir_Noteebook_a_PDF_en_Google_Colab.ipynb

# Convierte a HTML
!jupyter nbconvert Regresión_Lineal_Múltiple.ipynb --to html

# Instala paquetes para crear el pdf
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

# Convierte el Notebook en PDF
!jupyter nbconvert Regresión_Lineal_Múltiple.ipynb --to pdf
```



In [ ]:
# Instale nbconvert

!jupyter nbconvert --to html /content/Modelación_hacia_adelante_y_PDF.ipynb

In [ ]:
# Convierte a HTML

!jupyter nbconvert Modelación_hacia_adelante_y_PDF.ipynb --to html

In [ ]:
# Instala paquetes para crear el pdf

!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

In [ ]:
# Convierte el Notebook en PDF

!jupyter nbconvert Modelación_hacia_adelante_y_PDF.ipynb --to pdf